# Plot the derivatives

### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

In [ ]:
data = np.loadtxt("scvh_hhe_y0.275_dt_cgs_lowrhot.csv", delimiter=",", skiprows=0) 

logT_table   = data[:, 0] 
logrho_table = data[:, 1]
logP_table   = data[:, 2]
logu_table   = data[:, 3]
logs_table   = data[:, 4]

# All SCVH EOS tables obtained from Ravit have the same size
#nRho = 201
#nT   = 100
nRho = 80
nT   = 48

logrho_table = logrho_table[0:nRho]
logT_table = logT_table[0:np.size(logT_table):nRho]

logrho_min = np.min(logrho_table)
logrho_max = np.max(logrho_table)
logT_min   = np.min(logT_table)
logT_max   = np.max(logT_table)
    
dlogrho = logrho_table[1:]-logrho_table[:-1]
dlogT = logT_table[1:]-logT_table[:-1]

print("logrho_min = {:}".format(logrho_min))
print("logrho_max = {:}".format(logrho_max))
print("logT_min   = {:}".format(logT_min))
print("logT_max   = {:}".format(logT_max))
print()
#print("logrho_table = {:}".format(logrho_table))
#print("logT_table   = {:}".format(logT_table))
#print()

# Mark where the original SCvH EOS table was extended below T=100K
index = np.min(np.where(logT_table>=2.0))
print("The EOS table was extended until index={:} logT={:}".format(index, logT_table[index]))

# Split into arrays of constant T
logP_array = np.split(logP_table, nT)
logu_array = np.split(logu_table, nT)
logs_array = np.split(logs_table, nT)

# Generate 2d arrays
logP = np.vstack(logP_array)
logu = np.vstack(logu_array)
logs = np.vstack(logs_array)

In [ ]:
# Normal variables
T_table = 10.0**logT_table
rho_table = 10.0**logrho_table

P = 10.0**logP
u = 10.0**logu
s = 10.0**logs

rho_min = 10.0**logrho_min
rho_max = 10.0**logrho_max
T_min = 10.0**logT_min
T_max = 10.0**logT_max

print("rho_min = {:}".format(rho_min))
print("rho_max = {:}".format(rho_max))
print("T_min   = {:}".format(T_min))
print("T_max   = {:}".format(T_max))
print()


Here we generate interpolation function from ```scipy.interp``` to obtain an independent calculation of the derivatives. Since the ```scipy.interp.interp2d``` does not provide derivatives we use the $1D$ interpolation functions. These also support *Akima splines* which have many desirable properties.

In [ ]:
# Generate interpolation functions to calculate the derivatives
logP_table_int_rho = list()
logP_table_int_T   = list()
logu_table_int_rho = list()
logu_table_int_T   = list()
logs_table_int_rho = list()
logs_table_int_T   = list()

dlogP_dlogrho_int_rho = list()
dlogP_dlogT_int_T = list()

dlogu_dlogrho_int_rho = list()
dlogu_dlogT_int_T = list()

dlogs_dlogrho_int_rho = list()
dlogs_dlogT_int_T = list()

# Interpolation for T=const.
# Akima splines
"""
for i in range(0, nT):
    logP_table_int_rho.append(interp.Akima1DInterpolator(logrho_table, logP[i,:]))
    logu_table_int_rho.append(interp.Akima1DInterpolator(logrho_table, logu[i,:]))
    logs_table_int_rho.append(interp.Akima1DInterpolator(logrho_table, logs[i,:]))
"""
# Linear interpolation
for i in range(0, nT):
    logP_table_int_rho.append(interp.InterpolatedUnivariateSpline(logrho_table, logP[i,:], k=1))
    logu_table_int_rho.append(interp.InterpolatedUnivariateSpline(logrho_table, logu[i,:], k=1))
    logs_table_int_rho.append(interp.InterpolatedUnivariateSpline(logrho_table, logs[i,:], k=1))

# Calculate first derivatives
for i in range(0, nT):
    dlogP_dlogrho_int_rho.append(logP_table_int_rho[i].derivative())
    dlogu_dlogrho_int_rho.append(logu_table_int_rho[i].derivative())
    dlogs_dlogrho_int_rho.append(logs_table_int_rho[i].derivative())
    
# Interpolation for rho=const.
"""
# Akima splines
for i in range(0, nRho):
    logP_table_int_T.append(interp.Akima1DInterpolator(logT_table, logP[:,i]))
    logu_table_int_T.append(interp.Akima1DInterpolator(logT_table, logu[:,i]))
    logs_table_int_T.append(interp.Akima1DInterpolator(logT_table, logs[:,i]))
"""
# Linear interpolation
for i in range(0, nRho):
    logP_table_int_T.append(interp.InterpolatedUnivariateSpline(logT_table, logP[:,i], k=1))
    logu_table_int_T.append(interp.InterpolatedUnivariateSpline(logT_table, logu[:,i], k=1))
    logs_table_int_T.append(interp.InterpolatedUnivariateSpline(logT_table, logs[:,i], k=1))

# Calculate first derivatives
for i in range(0, nRho):
    dlogP_dlogT_int_T.append(logP_table_int_T[i].derivative())
    dlogu_dlogT_int_T.append(logu_table_int_T[i].derivative())
    dlogs_dlogT_int_T.append(logs_table_int_T[i].derivative())


Calculate the normal derivatives, e.g.,
$$ \left. \frac{\partial P}{\partial \rho} \right|_{T} = \frac{P}{\rho} \left. \frac{\partial \log P}{\partial \log \rho} \right|_{\log T}   $$
$$ \left. \frac{\partial P}{\partial T} \right|_{\rho} = \frac{P}{T} \left. \frac{\partial \log P}{\partial \log T} \right|_{\log \rho}   $$
from ```scipy.interp```.


In [ ]:
#### This is probably not useful
dP_drho_int_rho = list()
dP_dT_int_T = list()

du_drho_int_rho = list()
du_dT_int_T = list()

ds_drho_int_rho = list()
ds_dT_int_T = list()

# Derivatives for T=const.
for i in range(0, nT):
    dlogP_dlogrho_int_rho.append(logP_table_int_rho[i].derivative())
    dlogu_dlogrho_int_rho.append(logu_table_int_rho[i].derivative())
    dlogs_dlogrho_int_rho.append(logs_table_int_rho[i].derivative())
    
# Interpolation for rho=const.
"""
# Akima splines
for i in range(0, nRho):
    logP_table_int_T.append(interp.Akima1DInterpolator(logT_table, logP[:,i]))
    logu_table_int_T.append(interp.Akima1DInterpolator(logT_table, logu[:,i]))
    logs_table_int_T.append(interp.Akima1DInterpolator(logT_table, logs[:,i]))
"""
# Linear interpolation
for i in range(0, nRho):
    logP_table_int_T.append(interp.InterpolatedUnivariateSpline(logT_table, logP[:,i], k=1))
    logu_table_int_T.append(interp.InterpolatedUnivariateSpline(logT_table, logu[:,i], k=1))
    logs_table_int_T.append(interp.InterpolatedUnivariateSpline(logT_table, logs[:,i], k=1))

# Calculate first derivatives
for i in range(0, nRho):
    dlogP_dlogT_int_T.append(logP_table_int_T[i].derivative())
    dlogu_dlogT_int_T.append(logu_table_int_T[i].derivative())
    dlogs_dlogT_int_T.append(logs_table_int_T[i].derivative())


In [ ]:
# Load derivatives
logrho_int = np.loadtxt("testscvheosinterp_rhoaxis.txt")
logT_int = np.loadtxt("testscvheosinterp_taxis.txt")

dlogPdlogrho_int = np.loadtxt("testscvheosinterp_dlogpdlogrho.txt")
dlogPdlogT_int = np.loadtxt("testscvheosinterp_dlogpdlogt.txt")

dlogudlogrho_int = np.loadtxt("testscvheosinterp_dlogudlogrho.txt")
dlogudlogT_int = np.loadtxt("testscvheosinterp_dlogudlogt.txt")

dlogsdlogrho_int = np.loadtxt("testscvheosinterp_dlogsdlogrho.txt")
dlogsdlogT_int = np.loadtxt("testscvheosinterp_dlogsdlogt.txt")

nRho_int = np.size(logrho_int)
nT_int = np.size(logT_int)

print("nRho_int= {:} nT_int= {:}".format(nRho_int, nT_int))


In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 4
nSkipRho = 8

nSkipT = 1
nSkipRho = 1

First we verify that the interpolation functions from ```scipy.interp``` produce sensible results.

In [ ]:
# Plot log(P), log(u) and log(s) to verify that the python interpolation functions work
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logrho_table, logP[i,:], '-')
    ax[0][0].plot(logrho_int, logP_table_int_rho[i](logrho_int), '--')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H-He, T=const.)")

# P(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, logP[:,i], '-')
    ax[0][1].plot(logT_int, logP_table_int_T[i](logT_int), '--')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H-He (rho=const.)")

# u(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, logu[i,:], '-')
    ax[1][0].plot(logrho_int, logu_table_int_rho[i](logrho_int), '--')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, logu[:,i], '-')
    ax[1][1].plot(logT_int, logu_table_int_T[i](logT_int), '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, nT, nSkipT):
    ax[2][0].plot(logrho_table, logs[i,:], '-')
    ax[2][0].plot(logrho_int, logs_table_int_rho[i](logrho_int), '--')
    
ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, nRho, nSkipRho):
    ax[2][1].plot(logT_table, logs[:,i], '-')
    ax[2][1].plot(logT_int, logs_table_int_T[i](logT_int), '--')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


Now we plot the derivatives obtained with ```scipy.interp``` and compare to the results from ```testscvheosderivs.c```.

In [ ]:
# Plot the derivatives
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# dlogP/dlogrho(rho, T=const)
ax[0][0].set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logrho_table, dlogP_dlogrho_int_rho[i](logrho_table), '-')

for i in range(0, nT_int, nSkipT):
    ax[0][0].plot(logrho_int, dlogPdlogrho_int[i,:], '--')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="dlog(P)/dlog(rho)")
ax[0][0].set(title="SCvH EOS H-He, T=const.)")

# dlogP/dlogT(rho=const, T)
ax[0][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[0][1].plot(logT_table, dlogP_dlogT_int_T[i](logT_table), '-')

for i in range(0, nRho_int, nSkipRho):
    ax[0][1].plot(logT_int, dlogPdlogT_int[:,i], '--')

ax[0][1].set(xlabel="log(T) [K]", ylabel="dlog(P)/dlog(T)")
ax[0][1].set(title="SCvH EOS H-He (rho=const.)")

# dlogu/dlogrho(rho, T=const)
ax[1][0].set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logrho_table, dlogu_dlogrho_int_rho[i](logrho_table), '-')

for i in range(0, nT_int, nSkipT):
    ax[1][0].plot(logrho_int, dlogudlogrho_int[i,:], '--')
    
ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="dlog(u)/dlog(rho)")
ax[1][0].set(title="SCvH EOS H-He, T=const.)")

# dlogu/dlogT(rho=const, T)
ax[1][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[1][1].plot(logT_table, dlogu_dlogT_int_T[i](logT_table), '-')

for i in range(0, nRho_int, nSkipRho):
    ax[1][1].plot(logT_int, dlogudlogT_int[:,i], '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="dlog(u)/dlog(T)")
ax[1][1].set(title="SCvH EOS H-He (rho=const.)")

# dlogs/dlogrho(rho, T=const)
ax[2][0].set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[2][0].plot(logrho_table, dlogs_dlogrho_int_rho[i](logrho_table), '-')

for i in range(0, nT_int, nSkipT):
    ax[2][0].plot(logrho_int, dlogsdlogrho_int[i,:], '--')
    
ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="dlog(s)/dlog(rho)")

# dlogs/dlogT(rho=const, T)
ax[2][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[2][1].plot(logT_table, dlogs_dlogT_int_T[i](logT_table), '-')

for i in range(0, nRho_int, nSkipRho):
    ax[2][1].plot(logT_int, dlogsdlogT_int[:,i], '--')

ax[2][1].set(xlabel="log(T) [K]", ylabel="dlog(s)/dlog(T)")
ax[2][1].set(title="SCvH EOS H-He (rho=const.)")


Finally we plot the derivatives in normal variables with ```scipy.interp``` and compare to the results from ```testscvheosderivs.c```.

In [ ]:
# Plot the derivatives
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# dP/drho(rho, T=const)
ax[0][0].set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[0][0].semilogx(rho_table, P[i,:]/rho_table[i]*dlogP_dlogrho_int_rho[i](np.log10(rho_table)), '-')

#for i in range(0, nT_int, nSkipT):
#    ax[0][0].plot(logrho_int, dlogPdlogrho_int[i,:], '--')
    
ax[0][0].set(xlabel="rho [g cm^${-3}$]", ylabel="dP/drho")
ax[0][0].set(title="SCvH EOS H-He, T=const.)")

# dP/dT(rho=const, T)
ax[0][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[0][1].semilogx(T_table, P[:,i]/T_table*dlogP_dlogT_int_T[i](np.log10(T_table)), '-')

#for i in range(0, nRho_int, nSkipRho):
#    ax[0][1].plot(logT_int, dlogPdlogT_int[:,i], '--')

ax[0][1].set(xlabel="T [K]", ylabel="dP/dT")
ax[0][1].set(title="SCvH EOS H-He (rho=const.)")

# du/drho(rho, T=const)
ax[1][0].set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    #ax[1][0].plot(logrho_table, dlogu_dlogrho_int_rho[i](logrho_table), '-')
    ax[1][0].semilogx(rho_table, u[i,:]/rho_table[i]*dlogu_dlogrho_int_rho[i](np.log10(rho_table)), '-')


#for i in range(0, nT_int, nSkipT):
#    ax[1][0].plot(logrho_int, dlogudlogrho_int[i,:], '--')
    
ax[1][0].set(xlabel="rho [g cm^${-3}$]", ylabel="du/drho")
ax[1][0].set(title="SCvH EOS H-He, T=const.)")

# du/dT(rho=const, T)
ax[1][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[1][1].semilogx(T_table, u[:,i]/T_table*dlogu_dlogT_int_T[i](np.log10(T_table)), '-')

#for i in range(0, nRho_int, nSkipRho):
#    ax[1][1].plot(logT_int, dlogudlogT_int[:,i], '--')

ax[1][1].set(xlabel="T [K]", ylabel="du/dT")
ax[1][1].set(title="SCvH EOS H-He (rho=const.)")

# ds/drho(rho, T=const)
ax[2][0].set_prop_cycle(None)
for i in range(0, nT, nSkipT):
    ax[2][0].semilogx(rho_table, s[i,:]/rho_table[i]*dlogs_dlogrho_int_rho[i](np.log10(rho_table)), '-')

#for i in range(0, nT_int, nSkipT):
#    ax[2][0].plot(logrho_int, dlogsdlogrho_int[i,:], '--')
    
ax[2][0].set(xlabel="rho [g cm^${-3}$]", ylabel="ds/drho")

# ds/dT(rho=const, T)
ax[2][1].set_prop_cycle(None)

for i in range(0, nRho, nSkipRho):
    ax[2][1].semilogx(T_table, s[:,i]/T_table*dlogs_dlogT_int_T[i](np.log10(T_table)), '-')

#for i in range(0, nRho_int, nSkipRho):
#    ax[2][1].plot(logT_int, dlogsdlogT_int[:,i], '--')

ax[2][1].set(xlabel="T [K]", ylabel="ds/dT")
ax[2][1].set(title="SCvH EOS H-He (rho=const.)")
